In [1]:
pip install selenium webdriver-manager

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\duong\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import csv
import time
import random

# URL trang bắt đầu
START_URL = "https://batdongsan.com.vn/ban-can-ho-chung-cu"

# ✅ Cập nhật đường dẫn tới Chrome profile thật của bạn (chỉnh lại đúng đường dẫn máy bạn)
PROFILE_PATH = r"C:\Users\duong\AppData\Local\Google\Chrome\User Data"
PROFILE_NAME = "Default"  # hoặc Profile 1, Profile 2, tùy máy

def init_driver():
    options = Options()
    options.add_argument(f"--user-data-dir={PROFILE_PATH}")
    options.add_argument(f"--profile-directory={PROFILE_NAME}")
    options.add_argument("--start-maximized")
    options.add_argument("--disable-blink-features=AutomationControlled")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    return driver

def crawl_page(driver, url):
    driver.get(url)

    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "article.js__card"))
        )
    except:
        print("⚠ Không load được bài đăng.")
        return []

    time.sleep(random.uniform(2, 3))

    cards = driver.find_elements(By.CSS_SELECTOR, "article.js__card")
    data = []

    for card in cards:
        try:
            title = card.find_element(By.CSS_SELECTOR, "h3").text.strip()
            link = card.find_element(By.TAG_NAME, "a").get_attribute("href").strip()
            price = card.find_element(By.CLASS_NAME, "js__card-config-price").text.strip()
            area = card.find_element(By.CLASS_NAME, "js__card-config-area").text.strip()
            location = card.find_element(By.CLASS_NAME, "js__card-location").text.strip()

            print(f"📄 Crawling: {title}")
            description = get_detail_description(driver, link)

            data.append({
                "title": title,
                "price": price,
                "area": area,
                "location": location,
                "description": description,
                "link": link
            })
        except Exception as e:
            print("❗ Bỏ qua bài bị lỗi:", e)
            continue

    return data

def get_detail_description(driver, url):
    try:
        driver.get(url)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "re__section-body"))
        )
        time.sleep(random.uniform(1.5, 2.5))
        desc_elem = driver.find_element(By.CLASS_NAME, "re__section-body")
        return desc_elem.text.strip()
    except Exception:
        return ""

def save_to_csv(data, filename="bds_data_full_selenium.csv"):
    keys = ["title", "price", "area", "location", "description", "link"]
    with open(filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=keys)
        writer.writeheader()
        writer.writerows(data)
    print(f"✅ Đã lưu {len(data)} bài viết vào {filename}")

def main():
    driver = init_driver()
    all_data = []

    for page in range(1, 6):  # Crawl 5 trang đầu
        print(f"\n🔎 Crawl trang {page}...")
        url = f"{START_URL}/p{page}"
        page_data = crawl_page(driver, url)
        all_data.extend(page_data)

    save_to_csv(all_data)
    driver.quit()

if __name__ == "__main__":
    main()


SessionNotCreatedException: Message: session not created: Chrome failed to start: crashed.
  (session not created: DevToolsActivePort file doesn't exist)
  (The process started from chrome location C:\Program Files\Google\Chrome\Application\chrome.exe is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
	GetHandleVerifier [0x0xd33b03+62899]
	GetHandleVerifier [0x0xd33b44+62964]
	(No symbol) [0x0xb610f3]
	(No symbol) [0x0xb9638a]
	(No symbol) [0x0xb903e9]
	(No symbol) [0x0xbda36a]
	(No symbol) [0x0xbd9c0a]
	(No symbol) [0x0xbce306]
	(No symbol) [0x0xb9d670]
	(No symbol) [0x0xb9e4e4]
	GetHandleVerifier [0x0xf94793+2556483]
	GetHandleVerifier [0x0xf8fd02+2537394]
	GetHandleVerifier [0x0xd5a2fa+220586]
	GetHandleVerifier [0x0xd4aae8+157080]
	GetHandleVerifier [0x0xd5141d+184013]
	GetHandleVerifier [0x0xd3ba68+95512]
	GetHandleVerifier [0x0xd3bc10+95936]
	GetHandleVerifier [0x0xd26b5a+9738]
	BaseThreadInitThunk [0x0x77265d49+25]
	RtlInitializeExceptionChai